In [1]:
from flask import Flask, request, jsonify
import requests
import json
import asyncio

app = Flask(__name__)

async def fetch_numbers(url):
    try:
        response = await asyncio.wait_for(requests.get(url), timeout=5)
        data = response.json()
        return data.get("numbers", [])
    except (requests.RequestException, json.JSONDecodeError, asyncio.TimeoutError):
        return []

@app.route('/numbers', methods=['GET'])
def get_numbers():
    urls = request.args.getlist('url')
    loop = asyncio.get_event_loop()
    tasks = [fetch_numbers(url) for url in urls]
    results = loop.run_until_complete(asyncio.gather(*tasks))
    
    merged_numbers = list(set(number for sublist in results for number in sublist))
    merged_numbers.sort()
    
    return jsonify({"numbers": merged_numbers})

if __name__ == '__main__':
    app.run(host='localhost', port=8008)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8008
Press CTRL+C to quit
127.0.0.1 - - [14/Aug/2023 19:15:02] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [14/Aug/2023 19:15:03] "GET /favicon.ico HTTP/1.1" 404 -
